# Computing Krill NASC from Ship Echosounder Data

Jupyter notebook accompanying the manuscript:

_Echopype: A Python library for interoperable and scalable processing of ocean sonar data for biological information_   
_Authors: Wu-Jung Lee, Landung Setiawan, Caesar Tuguinay, Emilio Mayorga, and Valentina Staneva_

## Introduction

### Description

This notebook uses a 4-day subset of the EK60 echosounder data collected during the [2017 Joint U.S.-Canada Integrated Ecosystem and Pacific Hake Acoustic Trawl Survey (aka the "Hake survey")](https://www.fisheries.noaa.gov/west-coast/science-data/joint-us-canada-integrated-ecosystem-and-pacific-hake-acoustic-trawl-survey) to illustrate a processing pipeline that compute Nautical Areal Scattering Coefficient (NASC) attributed to krill. The workflow includes steps to perform data conversion, calibration, regridding, and frequency differencing, using functions from [`Echopype`](https://echopype.readthedocs.io) and core scientific Python software packages, particularly `NumPy`, `Xarray`, and `Dask`.

### Outline

1) Convert EK60 `.raw` files to `EchoData` objects
2) Combine `EchoData` objects
3) Calibrate raw backscatter measurement in the combined `EchoData` object to Sv
4) Regrid calibrated Sv data to MVBS
5) Generate and apply a frequency-differencing mask for krill
6) Compute water column-integreated NASC for krill
7) Plot water column-integrated krill NASC on the map

### Running the notebook

This notebook can be run with a conda environment created using the conda environment file https://github.com/OSOceanAcoustics/echopype-examples/blob/main/conda/environment-ep-main.yml. The notebook creates a directory, if not already present: `./exports/notebook_krill_outputs`. All Zarr files will be exported there.

### Note
We encourage importing `echopype` as `ep` for consistency.

In [1]:
from pathlib import Path
import os

import fsspec
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from dask.distributed import Client

import echopype as ep
from echopype import colormap # needed for the EK500 colormap

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
# Ignore large graph dask UserWarnings
warnings.simplefilter("ignore", category=UserWarning)

### Establish AWS S3 file system connection and generate list of target EK60 `.raw` files

Access and inspect the publicly accessible NCEI WCSD S3 bucket on the AWS cloud as if it were a local file system. This will be done through the Python [fsspec](https://filesystem-spec.readthedocs.io) file system and bytes storage interface. We will use `fsspec.filesystem.glob` (`fs.glob`) to generate a list of all EK60 `.raw` data files in the bucket, then filter on file names for target dates of interest. 

The directory path on the [ncei-wcsd-archive S3 bucket](https://ncei-wcsd-archive.s3.amazonaws.com/index.html) is `s3://ncei-wcsd-archive/data/raw/Bell_M._Shimada/SH1707/EK60/`. All `.raw` files from the 2017 Hake survey cruise are found here.

In [2]:
# Setup filesystem
fs = fsspec.filesystem('s3', anon=True)
bucket = "ncei-wcsd-archive"
rawdirpath = "data/raw/Bell_M._Shimada/SH1707/EK60"

# Select all RAW files from July 2017
s3rawfiles = fs.glob(f"{bucket}/{rawdirpath}/*.raw")
select_files = [
    s3path
    for s3path in s3rawfiles
    if any(
        f"D2017{datestr}" in s3path
        for datestr in ["0726"]#, "0727", "0728", "0729"]
    )
]
print(f"There are {len(select_files)} target raw files available")

There are 70 target raw files available


These RAW files combined are about 8.1 GiB in size.

**NOTE:**

If you are on Binder or are running this on a machine with less than 8 GiB of RAM, we recommend you to use only a subset of the `select_files`.

In [3]:
# Uncomment the line below to use a subset of the 324 files

# select_files = select_files[0:10]

Most of the data below will be stored on disk, since it cannot fit in memory. Please modify the `base_path` below to point to disk storage location where you have **at least 50 GiB** of storage space available.

Create directories to store Zarr files:

In [4]:
base_path = Path("./exports/notebook_krill_outputs")
base_path.mkdir(exist_ok=True, parents=True)

echodata_zarr_path = base_path / "echodata_zarr"
echodata_zarr_path.mkdir(exist_ok=True)
combined_zarr_path = base_path / "combined_zarr"
combined_zarr_path.mkdir(exist_ok=True)

### Dask Client Setup

Echopype leverages Dask's lazy-load mechanisms to perform out-of-core and parallelized computation on large datasets. We use the Dask `Client` that is pointed to a `Scheduler` that schedules tasks and allocate memory for these computations.

Due to the large volumes of data (if you did not subselect files above), if you have enough memory resources, set the `n_workers` to be such that each worker has at least 4 GiB of memory.

In [5]:
# Use maximum number of CPUs for Dask Client
client = Client() # Set n_workers so that total_RAM / n_workers >= 4
                  # or leave empty and let Dask decide
print("Dask Client Dashboard:", client.dashboard_link)

Dask Client Dashboard: http://127.0.0.1:8787/status


Click on the Dask Client Dashboard link above to view the computations live.

For more information on how to use, view, and interpret the Dashboard, go to this link: https://docs.dask.org/en/stable/dashboard.html

## Data processing workflow

The following data processing steps follow the typical Echopype workflow, which converts and standardizes data before performing more computation. Below we first convert and save the `.raw` files generated by EK60 to Zarr stores on disk, and lazy-load them back for downstream processing. This allows us to perform computations that may require resources that are larger than the local system memory.

### Convert EK60 `.raw` Files to `EchoData` Objects

In [6]:
%%time

# Save EchoData objects locally
def open_and_save(raw_file, sonar_model, use_swap, save_path, storage_options):
    try:
        ed = ep.open_raw(
            raw_file=f's3://{raw_file}',
            sonar_model=sonar_model,
            use_swap=use_swap,
            storage_options=storage_options,
        )
        ed.to_zarr(save_path, overwrite=True, compute=True)
    except Exception as e:
        print("Error with Exception: ", e)

# Parse EK60 `.RAW` file and save to Zarr Store
open_and_save_futures = []
for raw_file_url in select_files:
    open_and_save_future = client.submit(
        open_and_save,
        raw_file=raw_file_url,
        sonar_model='ek60',
        use_swap=True,
        save_path=echodata_zarr_path,
        storage_options={'anon': True}
    )
    open_and_save_futures.append(open_and_save_future)
open_and_save_futures = client.gather(open_and_save_futures)

CPU times: user 15.4 s, sys: 4.87 s, total: 20.3 s
Wall time: 7min 5s


### Combine Echodata Objects

Under the hood of the `ep.combine_echodata` function, the Zarr stores of each `EchoData` object are loaded and appended lazily, meaning that no data loading or appending actually happens until the combined object is saved to disk via `to_zarr`. Since the `EchoData` is appended sequentially based on the data time sequence and therefore not parallelized, this is the slowest stage of the processing pipeline.

In [1]:
%%time

# Open (lazy-load) Zarr stores containing EchoData Objects, and lazily combine them
ed_future_list = []
for converted_file in sorted(echodata_zarr_path.glob("*.zarr")):
    ed_future = client.submit(    
        ep.open_converted,
        converted_raw_path=converted_file,
        chunks={}
    )
    ed_future_list.append(ed_future)
ed_list = client.gather(ed_future_list)
ed_combined = ep.combine_echodata(ed_list)

# Save the combined EchoData object to a new Zarr store
# The appending operation only happens when relevant data needs to be save to disk
ed_combined.to_zarr(
    combined_zarr_path / "ed_combined.zarr",
    overwrite=True,
    compute=True,
)

# Lazy-load the comined Zarr store
ed_combined = ep.open_converted(
    combined_zarr_path / "ed_combined.zarr",
    chunks={},
)

NameError: name 'echodata_zarr_path' is not defined

### Calibrate Combined `EchoData` Object

Compute Volume Backscattering Strength (Sv) from raw backscatter measurements stored in the converted `EchoData` object, and add depth and latitude/longitude to this Xarray Dataset.

In [13]:
%%time

# Chunk Echodata Object ping-wise
ed_combined = ed_combined.chunk({"ping_time": 1000, "range_sample": -1})

# Compute Sv
ds_Sv = ep.calibrate.compute_Sv(ed_combined)

# Add Depth
ds_Sv = ep.consolidate.add_depth(ds_Sv, depth_offset=9.15)

# Add Latitude and Longitude
ds_Sv = ep.consolidate.add_location(ds_Sv, ed_combined)

# Save to Zarr and offload computation to disk
ds_Sv.to_zarr(
    combined_zarr_path / "ds_Sv.zarr",
    mode="w",
    compute=True,
)

# Lazy-load the Zarr store
ds_Sv = xr.open_dataset(
    combined_zarr_path / "ds_Sv.zarr",
    engine="zarr",
    chunks={},
)

CPU times: user 7.55 s, sys: 2.59 s, total: 10.1 s
Wall time: 26 s


### Regrid Calibrated Data

Compute Mean Volume Backscattering Strength (MVBS), which are binned averages of Sv (in the linear domain) across ping time and depth. This gets the data onto a common grid from the original ping times and depth sample size that may vary across ping and channel depending on the echosounder setting.

In [14]:
%%time

# Compute MVBS
ds_MVBS = ep.commongrid.compute_MVBS(
    ds_Sv,
    range_var="depth",
    range_bin='5m', # in meters
    ping_time_bin='20s', # in seconds
)

# Save to Zarr and offload computation to disk
ds_MVBS.to_zarr(
    combined_zarr_path / "ds_MVBS.zarr",
    mode="w",
    compute=True,
)

# Lazy-load the Zarr store
ds_MVBS = xr.open_dataset(
    combined_zarr_path / "ds_MVBS.zarr",
    engine="zarr",
    chunks={},
)

CPU times: user 10.8 s, sys: 3.59 s, total: 14.4 s
Wall time: 1min 7s


### Generate and Apply a Frequency-Differencing Mask for Krill on the Echogram

Frequency-differencing uses echo strength variation across frequency to classify the scatterers. The threshold of the difference of echo strength between observations from a pair of frequencies can be derived empirically or semi-empirically based on physics-based scattering models.

Here we use the thresholds from Phillips et al. (2022) to identify krill:
$$
16.2 \, \textrm{dB} \geq \textrm{MVBS}_\textrm{120kHz} - \textrm{MVBS}_\textrm{38kHz} \geq 10.0 \, \textrm{dB}
$$

**Reference:**

Phillips EM, Chu D, Gauthier S, Parker-Stetter SL, Shelton AO, Thomas RE. (2022).
Spatiotemporal variability of euphausiids in the California Current Ecosystem: insights from a recently developed time series. 
ICES Journal of Marine Science, 79(4): 1312–1326.
https://doi.org/10.1093/icesjms/fsac055

Typically the data is also thresholded to ensure that the frequency-differencing is applied on echogram pixels with reasonably high echo strength.

In [15]:
%%time

# MVBS threshold in dB
th_MVBS = -70

# Create copy of MVBS
ds_non_empty_MVBS = ds_MVBS.copy()

# Replace all Sv less -70 dB with NaN
ds_non_empty_MVBS["Sv"] = xr.where(ds_MVBS["Sv"] < th_MVBS, np.nan, ds_MVBS["Sv"])

CPU times: user 2.93 ms, sys: 14 ms, total: 16.9 ms
Wall time: 17 ms


And then we use the Echopype functions to generate and apply masks to isolate krill on the echogram:

In [16]:
%%time

# Compute compute both frequency differencing inequalities as separate masks
freq_diff_mask_1 = ep.mask.frequency_differencing(
    ds_non_empty_MVBS,
    freqABEq="120 kHz - 38 kHz < 16.2 dB"
)
freq_diff_mask_2 = ep.mask.frequency_differencing(
    ds_non_empty_MVBS,
    freqABEq="120 kHz - 38 kHz > 10.0 dB"
)

# Apply both frequency differencing masks
ds_freq_diff_MVBS = ep.mask.apply_mask(
    ds_non_empty_MVBS,
    [freq_diff_mask_1, freq_diff_mask_2]
)

CPU times: user 220 ms, sys: 46.3 ms, total: 266 ms
Wall time: 629 ms


### Compute Water Column-Integreated NASC for Krill

NASC is typically computed along regular distance "intervals" and depth "layers." Here, we use an interval of 0.5 nmi and a layer thickness of 10 m, and integrate the NASC values for the top 250 m of the water column.

In [17]:
%%time

# Compute NASC
ds_NASC = ep.commongrid.compute_NASC(
    # Grab only the first 250m in the MVBS
    ds_freq_diff_MVBS,
    range_bin="10 m",
    dist_bin="0.5 nmi"
)

CPU times: user 625 ms, sys: 88.8 ms, total: 714 ms
Wall time: 1.05 s


In [18]:
# Find the 120 kHz channel
ch_sel = [True if "120" in ch_id else False for ch_id in ds_NASC["channel"].values]

In [19]:
# Integrate NASC at 120 kHz over the top 250 m of water column
ds_NASC_int_120kHz = (
    ds_NASC
    .sel(depth=slice(0, 250)).isel(channel=ch_sel)
    .squeeze().drop("channel")
) 
ds_NASC_int_120kHz["NASC"] = (
    ds_NASC_int_120kHz["NASC"].sum(dim="depth") * 10
)


In [20]:
# Add attributes for automatic xarray plotting
ds_NASC_int_120kHz["NASC"].attrs = {
    "long_name": "Nautical Areal Scattering Coefficient (NASC)",
    "units": "m2 nmi-2"
}


### Plot Water Column Krill dB NASC on map

For plotting, let's plot the NASC values in the log domain so that we can see both the high and low values easily.

In [2]:
%%time

# Log-transform NASC values
ds_NASC_int_120kHz["NASC_log"] = 10 * np.log10(ds_NASC_int_120kHz["NASC"]).compute()
ds_NASC_int_120kHz["NASC_log"].attrs = {
    "long_name": "Log of NASC",
    "units": "m2 nmi-2"
}

NameError: name 'np' is not defined

In the visualization below, the empty water columns (encoded as `NaN` in the NASC dataset) will be drawn with a translucent circle, and we use a dB scale to visualize the wide range of values in NASC:

In [3]:
%%time

# Create plot
fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': ccrs.PlateCarree()})

# Add coastlines and gridlines
ax.coastlines()
gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1.5,
    color='gray',
    linestyle='--'
)
gl.top_labels = False
gl.bottom_labels = True
gl.left_labels = True
gl.right_labels = False

# Set lat lon extent
ax.set_extent([
    ds_NASC_int_120kHz["longitude"].min().values - 0.3,
    ds_NASC_int_120kHz["longitude"].max().values + 0.3,
    ds_NASC_int_120kHz["latitude"].min().values - 0.3,
    ds_NASC_int_120kHz["latitude"].max().values + 0.3,
], crs=ccrs.PlateCarree())

# Add land and ocean features
ax.add_feature(cfeature.LAND, color='green')
ax.add_feature(cfeature.OCEAN, color='lightblue')

# Plot dB NASC on map
p = ds_NASC_int_120kHz.plot.scatter(
    x="longitude",
    y="latitude",
    hue="NASC_log",
    ax=ax,
    transform=ccrs.PlateCarree(),
    linewidths=0.2,
    cmap="Reds"
)

# Set axes title
p.axes.set_title("Krill NASC at 120kHz", fontsize=16)

# Show plot
plt.show()

NameError: name 'plt' is not defined

## Package versions

In [23]:
import datetime
import dask
print(f"echopype: {ep.__version__}, xarray: {xr.__version__}, dask: {dask.__version__}")

print(f"\n{datetime.datetime.utcnow()} +00:00")

echopype: 0.8.5.dev82+gf9c1b27, xarray: 2024.6.0, dask: 2024.7.1

2024-07-27 04:58:14.832113 +00:00
